In [4]:
import numpy as np
import time
import datetime
import torch
import torch.nn as nn

In [2]:
def quant_cross(y_pred):
        if len(np.shape(y_pred)) == 3:
            crosscount = 0
            for i in range(np.shape(y_pred)[0]):
                for j in range(np.shape(y_pred)[1]):
                    for k in range(np.shape(y_pred)[2]-1):
                        if y_pred[i,j,k+1] < y_pred[i,j,k]:
                            crosscount = crosscount + 1 

        else:
            crosscount = 0
            for i in range(np.shape(y_pred)[0]):
                for j in range(np.shape(y_pred)[1]-1):
                    if y_pred[i,j+1] < y_pred[i,j]:
                        crosscount = crosscount + 1 

        crossrate = crosscount/(np.size(y_pred)-np.size(y_pred[...,0]))

        return crossrate


In [3]:
quants = np.array([[1,2,3],[3,2,1]])

quant_cross(quants)

0.5

In [4]:
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()
print(est)

1


In [4]:
datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H-%M-%S')

'2024-03-11_13-13-15'

In [5]:
RMSE_train = np.load('log/2024-03-11_13-47-44/train_stats/RMSE_train.npz', allow_pickle=True)

In [11]:
RMSE_train

NpzFile 'log/2024-03-11_13-47-44/train_stats/RMSE_train.npz' with keys: values, means, mas, times

In [12]:

means = RMSE_train['mas']
np.size(means)
means

array([3.53125024, 3.65625012, 4.15625008, ..., 4.48533434, 4.48463026,
       4.48589563])

In [1]:
A=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]
A[:12]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [4]:
A=np.array([500,500,500,500])
B=np.array([0,0,2000,0])

print(np.mean(np.abs(A)))
print(np.mean(np.abs(B)))
print(np.sqrt(np.mean(A**2)))
print(np.sqrt(np.mean(B**2)))

500.0
500.0
500.0
1000.0


In [7]:
#Set up NN structure
no_nodes = 100

sequence= nn.Sequential(
    nn.Linear(13,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, 3*12) #Output dimesion is number of quantiles times number of target variables
)

print(sequence)

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=100, bias=True)
  (7): ReLU()
  (8): Linear(in_features=100, out_features=100, bias=True)
  (9): ReLU()
  (10): Linear(in_features=100, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=36, bias=True)
)


In [8]:
no_layers=5
modules = []
modules.append(nn.Linear(len(X_cols),no_nodes))
modules.append(nn.ReLU())
for i in range(no_layers):
    modules.append(nn.Linear(no_nodes,no_nodes))
    modules.append(nn.ReLU())
modules.append(nn.Linear(no_nodes, len(quantiles)*len(y_cols)))
sequence = lambda: nn.Sequential(*modules)

print(sequential)

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=100, bias=True)
  (7): ReLU()
  (8): Linear(in_features=100, out_features=100, bias=True)
  (9): ReLU()
  (10): Linear(in_features=100, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=36, bias=True)
)


In [40]:
import numpy as np
uncs = np.array([1,5,10])
invuncs_norm = (1/uncs)/sum(1/uncs)
print(invuncs_norm)
preds = np.array([1,2,3])
uncpreds = invuncs_norm*preds

prediction = sum(uncpreds)
print(uncpreds)
print(np.mean(preds))
print(prediction)


[0.76923077 0.15384615 0.07692308]
[0.76923077 0.30769231 0.23076923]
2.0
1.3076923076923077


In [3]:
if 0:
    print('hello')

In [27]:
import numpy as np
import torch
import torch.nn as nn

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

fout = np.array([[[-9,-8,-7,-6,-5],[-9,-8,-7,-6,-5]],[[-9,-8,-7,-6,-5],[-9,-8,-7,-6,-5]]])

#fout = fo.reshape((-1, 2, 3))

fout=torch.tensor(fout,dtype=torch.float,device=device)
softplus = nn.Softplus()
print(softplus)
print(torch.cat((fout[...,0:1], fout[...,0:1] + torch.cumsum(softplus(fout[...,1:]), dim=-1)), dim=-1))
print(torch.cumsum(softplus(fout), dim=-1))
print(softplus(fout))


CUDA is available. Using GPU.
Tensor is on device: cuda:0
Softplus(beta=1, threshold=20)
tensor([[[-9.0000, -8.9997, -8.9988, -8.9963, -8.9896],
         [-9.0000, -8.9997, -8.9988, -8.9963, -8.9896]],

        [[-9.0000, -8.9997, -8.9988, -8.9963, -8.9896],
         [-9.0000, -8.9997, -8.9988, -8.9963, -8.9896]]], device='cuda:0')
tensor([[[0.0001, 0.0005, 0.0014, 0.0038, 0.0106],
         [0.0001, 0.0005, 0.0014, 0.0038, 0.0106]],

        [[0.0001, 0.0005, 0.0014, 0.0038, 0.0106],
         [0.0001, 0.0005, 0.0014, 0.0038, 0.0106]]], device='cuda:0')
tensor([[[0.0001, 0.0003, 0.0009, 0.0025, 0.0067],
         [0.0001, 0.0003, 0.0009, 0.0025, 0.0067]],

        [[0.0001, 0.0003, 0.0009, 0.0025, 0.0067],
         [0.0001, 0.0003, 0.0009, 0.0025, 0.0067]]], device='cuda:0')


In [33]:
import numpy as np
fo = np.array([[[1,2,3],[1,2,3]],[[1,2,3],[1,2,3]]])
print(fo.shape)
print(fo[:,:,0:1].shape)
print('hej')
print(fo[:,:,0].shape)

(2, 2, 3)
(2, 2, 1)
hej
(2, 2)


In [29]:
A=[1,2,3,4,5]
A[0:1]
A[0]

1

In [2]:
import numpy as np
A=np.array([[1,1],[3,3]])
B=np.array([1,1])

dists=np.sqrt(np.sum((A-B)**2,axis=1))
meandist=np.mean(dists)
print(dists)
print(meandist)

C=np.array([1,2,3,4,5])
print(np.clip(C,3))

[0.         2.82842712]
1.4142135623730951


TypeError: clip() missing 1 required positional argument: 'a_max'

In [8]:
import torch
import numpy as np

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device


X=np.array([1,2,3])
tX = torch.tensor(X,dtype=torch.float,device=device)
tn=tX.clone()
tb=np.array([1,1])
ta=torch.tensor(tb,dtype=torch.float,device=device)
tn[:2]=tn[:2]+ta

print(tX)

CUDA is available. Using GPU.
Tensor is on device: cuda:0
tensor([1., 2., 3.], device='cuda:0')
